In [1]:
# keep dependandies to absolute minimum
from math import sin
from math import cos
from math import degrees
from math import radians
from math import atan2

In [2]:
def flatten(_list):
    return [item for sublist in _list for item in sublist]

In [3]:
def abssum(series):
    s = 0.0
    for val in series:
        s = s + abs(val)
    return s

In [4]:
class vector2:
    def __init__(self, coords):
        self.coords = coords
    
    def __repr__(self):
        return f'{self.coords}'
        
    def _get_x(self):
        return self.coords[0]
    def _get_y(self):
        return self.coords[1]    
    X = property(fget = _get_x)
    Y = property(fget = _get_y)
    
    def rot(self, degA):
        phi = radians(degA)
        xprim = self.X * cos(phi) - self.Y * sin(phi)
        yprim = self.X * sin(phi) + self.Y * cos(phi)
        return vector2([xprim, yprim])

In [5]:
class vector3:
    def __init__(self, coords):
        self.coords = coords
    
    def __repr__(self):
        return f'{self.coords}'
    
    def _get_x(self):
        return self.coords[0]
    def _get_y(self):
        return self.coords[1]
    def _get_z(self):
        return self.coords[2]
    
    X = property(fget = _get_x)
    Y = property(fget = _get_y)
    Z = property(fget = _get_z)
    
    def dot(self, vector):
        return self.X * vector.X + self.Y * vector.Y + self.Z * vector.Z
    
    def cross(self, vector):
        return vector3([
            self.Y * vector.Z - self.Z * vector.Y, 
            self.Z * vector.X - self.X * vector.Z, 
            self.X * vector.Y - self.Y * vector.X, 
        ])
    
    def scalar(self, value):
        return vector3([self.X * value, self.Y * value, self.Z * value])
    
    def inv(self):
        return vector3([-self.X, -self.Y, -self.Z])
    def add(self, b):
        return vector3([self.X + b.X, self.Y + b.Y, self.Z + b.Z])
    def sub(self, b):
        return self.add(b.inv())
    
    def rot_U(self, degA):
        prim = vector2([self.X, self.Y]).rot(degA)
        return vector3([prim.X, prim.Y, self.Z])
    def rot_V(self, degA):
        prim = vector2([self.Z, self.X]).rot(degA)
        return vector3([prim.Y, self.Y, prim.X])
    def rot_W(self, degA):
        prim = vector2([self.Y, self.Z]).rot(degA)
        return vector3([self.X, prim.X, prim.Y])
    def unitX():
        return vector3([1,0,0])
    def unitY():
        return vector3([0,1,0])
    def unitZ():
        return vector3([0,0,1])
    def unitVectors():
        return [vector3.unitX(), vector3.unitY(), vector3.unitZ()]
    def RotationAroundAxis(self, axisU, theta):
        crossux = axisU.cross(self)
        dotux = axisU.dot(self)
        sincomp = crossux.scalar(sin(radians(theta)))
        coscomp = crossux.scalar(cos(radians(theta))).cross(axisU)
        return axisU.scalar(dotux).add(coscomp).add(sincomp)

In [6]:
class position:
    def __init__(self, coords):
        self.coords = coords
        
    def _get_x(self):
        return self.coords[0]
    def _get_y(self):
        return self.coords[1]
    def _get_z(self):
        return self.coords[2]
    def _get_u(self):
        return self.coords[3]
    def _get_v(self):
        return self.coords[4]
    def _get_w(self):
        return self.coords[5]
    
    def _get_pos3(self):
        return vector3([self.X, self.Y, self.Z])
    
    pos3 = property(fget = _get_pos3)
    X = property(fget = _get_x)
    Y = property(fget = _get_y)
    Z = property(fget = _get_z)
    U = property(fget = _get_u)
    V = property(fget = _get_v)
    W = property(fget = _get_w)
    
    def add(self, values):
        return position([a+b for a,b in zip(self.coords, values)])
    def add_X(self, value):
        return self.add([value, 0, 0, 0, 0, 0])
    def add_Y(self, value):
        return self.add([0, value, 0, 0, 0, 0])
    def add_Z(self, value):
        return self.add([0, 0, value, 0, 0, 0])
    def add_U(self, value):
        return self.add([0, 0, 0, value, 0, 0])
    def add_V(self, value):
        return self.add([0, 0, 0, 0, value, 0])
    def add_W(self, value):
        return self.add([0, 0, 0, 0, 0, value])
    
    def add_tlx(self, value):
        axw, axv, axu = self._rotated_units()
        newpos = self.pos3.add(axw.scalar(value))
        return position([newpos.X, newpos.Y, newpos.Z, self.U, self.V, self.W])
    def add_tly(self, value):
        axw, axv, axu = self._rotated_units()
        newpos = self.pos3.add(axv.scalar(value))
        return position([newpos.X, newpos.Y, newpos.Z, self.U, self.V, self.W])
    def add_tlz(self, value):
        axw, axv, axu = self._rotated_units()
        newpos = self.pos3.add(axu.scalar(value))
        return position([newpos.X, newpos.Y, newpos.Z, self.U, self.V, self.W])
    def add_tlu(self, value):
        units = self._rotated_units()
        axw, axv, axu = units
        units = [u.RotationAroundAxis(axu, value) for u in units]
        uval, vval, wval = position._UVW_from_units(units)
        return position([self.X, self.Y, self.Z, uval, vval, wval])
    def add_tlv(self, value):
        units = self._rotated_units()
        axw, axv, axu = units
        units = [u.RotationAroundAxis(axv, value) for u in units]
        uval, vval, wval = position._UVW_from_units(units)
        return position([self.X, self.Y, self.Z, uval, vval, wval])
    def add_tlw(self, value):
        units = self._rotated_units()
        axw, axv, axu = units
        units = [u.RotationAroundAxis(axw, value) for u in units]
        uval, vval, wval = position._UVW_from_units(units)
        return position([self.X, self.Y, self.Z, uval, vval, wval])
    
    def abs_from_local(self, local):
        axisW, axisV, axisU = vector3.unitVectors()
        rot1 = self.pos3
        rot1 = rot1.RotationAroundAxis(axisW, local.W)
        rot1 = rot1.RotationAroundAxis(axisV, local.V)
        rot1 = rot1.RotationAroundAxis(axisU, local.U)
        rot1 = rot1.add(local.pos3)

        units = self._rotated_units()
        units = [u.RotationAroundAxis(axisW, local.W) for u in units]
        units = [u.RotationAroundAxis(axisV, local.V) for u in units]
        units = [u.RotationAroundAxis(axisU, local.U) for u in units]
        uvw = position._UVW_from_units(units)
        return position([rot1.X, rot1.Y, rot1.Z, uvw[0], uvw[1], uvw[2]])
    
    def abs_in_local(self, local):
        axisW, axisV, axisU = vector3.unitVectors()
        rot1 = self.pos3
        rot1 = rot1.sub(local.pos3)
        rot1 = rot1.RotationAroundAxis(axisU, -local.U)
        rot1 = rot1.RotationAroundAxis(axisV, -local.V)
        rot1 = rot1.RotationAroundAxis(axisW, -local.W)

        units = self._rotated_units()
        units = [u.RotationAroundAxis(axisU, -local.U) for u in units]
        units = [u.RotationAroundAxis(axisV, -local.V) for u in units]
        units = [u.RotationAroundAxis(axisW, -local.W) for u in units]
        uvw = position._UVW_from_units(units)
        return position([rot1.X, rot1.Y, rot1.Z, uvw[0], uvw[1], uvw[2]])
    
    def local_to_local(self, localfrom, localto):
        res = self.abs_from_local(localfrom)
        res = res.abs_in_local(localto)
        return res
    
    def _UVW_from_units(units):
        uval = degrees(atan2(units[0].Y, units[0].X))
        units = [u.rot_U(-uval) for u in units]
        vval = degrees(atan2(units[2].X, units[2].Z))
        units = [u.rot_V(-vval) for u in units]
        wval = degrees(atan2(units[1].Z, units[1].Y))
        return [uval, vval, wval]
    def _rotated_units(self):
        units = vector3.unitVectors()
        units = [u.rot_W(self.W) for u in units]
        units = [u.rot_V(self.V) for u in units]
        units = [u.rot_U(self.U) for u in units]
        return units
    
    def __repr__(self):
        return f'X:{self.X}\tY:{self.Y}\tZ:{self.Z}\tU:{self.U}\tV:{self.V}\tW:{self.W}'

In [15]:
# sanity check
p = position([0,0,0,0,0,0])
p = p.add_tlv(3)
p = p.add_Z(-3)
p = p.add_tlu(5)
p = p.add_tlx(5)
p = p.add_tly(6)
p = p.add_tlz(7)
p = p.add_tlw(8)
p = p.add_tlz(-9)

error1 = [a-b for a, b in zip(p.coords, [4.24282, 7.66074, -5.13739, 5.00683, 2.98857, 8.2617])]
print(error1)
print(f"{abssum(error1)=}")
# e-5 is within Real resolution in Epson

[3.498844654004074e-06, -1.550070596323394e-06, -4.067942096064314e-06, -3.1248967315988807e-06, 3.710005985979592e-06, 4.612981653551174e-06]
abssum(error1)=2.056474171752143e-05


In [16]:
local = position([1,2,3,1,2,3])
pos = position([4,5,6,5,4,3])
res = pos.abs_from_local(local)
res

X:5.133501609698349	Y:6.751995238308427	Z:9.110049565896762	U:6.217030137488549	V:5.724944256171328	W:6.177141759241397

In [17]:
ref = [5.1335, 6.752, 9.11005, 6.21703, 5.72494, 6.17714]


error2 = [a-b for a, b in zip(res.coords, ref)]
print(error2)
print(f"{abssum(error2)=}")
# e-5 is within Real resolution in Epson

[1.60969834883673e-06, -4.76169157259676e-06, -4.3410323691261965e-07, 1.3748854854611636e-07, 4.256171328265168e-06, 1.7592413970035636e-06]
abssum(error2)=1.2958394432160958e-05


In [18]:
local = position([3,2,1,1,2,3])
pos = position([4,5,6,5,4,3])
res = pos.abs_in_local(local)
res

X:0.8770664561514683	Y:3.241446051091901	Z:4.870706512340343	U:3.882505190398298	V:2.2109187993805612	W:-0.13451390416710154

In [19]:
ref = [0.877066, 3.24145, 4.87071, 3.88251, 2.21092, -0.134514]
error3 = [a-b for a, b in zip(res.coords, ref)]
print(error3)
print(f"{abssum(error3)=}")
# e-5 is within Real resolution in Epson

[4.5615146826438746e-07, -3.948908098738713e-06, -3.487659657075426e-06, -4.8096017017940085e-06, -1.2006194389790892e-06, 9.583289845016552e-08]
abssum(error3)=1.399877326330179e-05


In [20]:
local1 = position([1, 2, 3, 1, 2, 3])
local2 = position([4, 5, 6, 5, 4, 3])
pos = position([6, 3, 4, 6, 2, 5])

In [21]:
res = pos.local_to_local(local1, local2)
res

X:2.997576604009769	Y:-0.31497050782513575	Z:1.1695925817439499	U:2.115721314615743	V:-0.2143769337054294	W:5.052751908757081

In [22]:
ref = [2.99758, -0.314971, 1.16959, 2.11572, -0.214377, 5.05275]

error4 = [a-b for a, b in zip(res.coords, ref)]
print(error4)
print(f"{abssum(error4)=}")
# e-5 is within Real resolution in Epson

[-3.395990231247481e-06, 4.921748642527923e-07, 2.5817439499586925e-06, 1.3146157429311245e-06, 6.629457061091415e-08, 1.9087570812814647e-06]
abssum(error4)=9.75957644028247e-06


In [25]:
#reiterate final results
print(f"Epson has limited resolution, so errors much larger than double presicion of Python is expected")
print(f"coordinate shift checks {abssum(error1)=}")
print(f"local check {abssum(error2)=}")
print(f"reverse check {abssum(error3)=}")
print(f"local to local check {abssum(error4)=}")

Epson has limited resolution, so errors much larger than double presicion of Python is expected
coordinate shift checks abssum(error1)=2.056474171752143e-05
local check abssum(error2)=1.2958394432160958e-05
reverse check abssum(error3)=1.399877326330179e-05
local to local check abssum(error4)=9.75957644028247e-06
